# Text Analysis from SICSS

### Setup 
#### Install packages
- *tidyverse*
    - *ggplot2* for plotting
    - *dyplr* for dataframe manipulation
    - *tidyr* for getting to tidy data
- *tidytext* 

In [1]:
# install.packages("tidytext")
# install.packages("tidyverse")

#### Load packages

In [8]:
library(tidytext)
library(dplyr)
library(ggplot2)

In [9]:
load(url("https://cbail.github.io/Trump_Tweets.Rdata"))
## if the file is not available, download a local copy
# load(file = "trumptweets.Rdata")


[1] "status_id"              "created_at"             "user_id"               
 [4] "screen_name"            "text"                   "source"                
 [7] "reply_to_status_id"     "reply_to_user_id"       "reply_to_screen_name"  
[10] "is_quote"               "is_retweet"             "favorite_count"        
[13] "retweet_count"          "hashtags"               "symbols"               
[16] "urls_url"               "urls_t.co"              "urls_expanded_url"     
[19] "media_url"              "media_t.co"             "media_expanded_url"    
[22] "media_type"             "ext_media_url"          "ext_media_t.co"        
[25] "ext_media_expanded_url" "ext_media_type"         "mentions_user_id"      
[28] "mentions_screen_name"   "lang"                   "quoted_status_id"      
[31] "quoted_text"            "retweet_status_id"      "retweet_text"          
[34] "place_url"              "place_name"             "place_full_name"       
[37] "place_type"             "country"                "country_code"          
[40] "geo_coords"             "coords_coords"          "bbox_coords"

##### Format data
**Tokenization** - the way you define a unit of analysis (e.g. words, sequence of words, sentence)

**Tidy text format** One row per token (word) with  column variables that have extra context (e.g. which tweet the word came from)

In [12]:
# preview trumptweets
head(trumptweets,5)

status_id,created_at,user_id,screen_name,text,source,reply_to_status_id,reply_to_user_id,reply_to_screen_name,is_quote,⋯,retweet_text,place_url,place_name,place_full_name,place_type,country,country_code,geo_coords,coords_coords,bbox_coords
<chr>,<dttm>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<list>,<list>,<list>
997577906007298048,2018-05-18 20:41:21,25073877,realDonaldTrump,"Just met with UN Secretary-General António Guterres who is working hard to “Make the United Nations Great Again.” When the UN does more to solve conflicts around the world, it means the U.S. has less to do and we save money. @NikkiHaley is doing a fantastic job! https://t.co/pqUv6cyH2z",Twitter for iPhone,NA,NA,NA,FALSE,⋯,NA,NA,NA,NA,NA,NA,NA,"NA, NA","NA, NA","NA, NA, NA, NA, NA, NA, NA, NA"
997573139663028224,2018-05-18 20:22:25,25073877,realDonaldTrump,America is a Nation that believes in the power of redemption. America is a Nation that believes in second chances - and America is a Nation that believes that the best is always yet to come! #PrisonReform https://t.co/Yk5UJUYgHN,Twitter for iPhone,NA,NA,NA,FALSE,⋯,NA,NA,NA,NA,NA,NA,NA,"NA, NA","NA, NA","NA, NA, NA, NA, NA, NA, NA, NA"
997568208369577985,2018-05-18 20:02:49,25073877,realDonaldTrump,RT @SteveForbesCEO: .@realDonaldTrump speech on drug costs pays immediate dividends. New @Amgen drug lists at 30% less than expected. Middl…,Twitter for iPhone,NA,NA,NA,FALSE,⋯,".@realDonaldTrump speech on drug costs pays immediate dividends. New @Amgen drug lists at 30% less than expected. Middlemen like Pharmacy Benefit Managers, insurers &amp; hospitals would do well by passing discounts on to patients. @SecAzar @SGottliebFDA https://t.co/mfRQ5COtev",NA,NA,NA,NA,NA,NA,"NA, NA","NA, NA","NA, NA, NA, NA, NA, NA, NA, NA"
997515759281680385,2018-05-18 16:34:24,25073877,realDonaldTrump,"We grieve for the terrible loss of life, and send our support and love to everyone affected by this horrible attack in Texas. To the students, families, teachers and personnel at Santa Fe High School – we are with you in this tragic hour, and we will be with you forever... https://t.co/LtJ0D29Hsv",Twitter for iPhone,NA,NA,NA,FALSE,⋯,NA,NA,NA,NA,NA,NA,NA,"NA, NA","NA, NA","NA, NA, NA, NA, NA, NA, NA, NA"
997493407097524224,2018-05-18 15:05:35,25073877,realDonaldTrump,School shooting in Texas. Early reports not looking good. God bless all!,Twitter for iPhone,NA,NA,NA,FALSE,⋯,NA,NA,NA,NA,NA,NA,NA,"NA, NA","NA, NA","NA, NA, NA, NA, NA, NA, NA, NA"


In [4]:
tidy_trump_tweets<- trumptweets %>%
    select(created_at,text) %>%
    unnest_tokens("word", text)
head(tidy_trump_tweets)

created_at,word
<dttm>,<chr>
2018-05-18 20:41:21,just
2018-05-18 20:41:21,met
2018-05-18 20:41:21,with
2018-05-18 20:41:21,un
2018-05-18 20:41:21,secretary
2018-05-18 20:41:21,general
